In [1]:
# Import required libraries for data manipulation and analysis
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import time
from scipy.cluster import hierarchy
from scipy.spatial.distance import squareform
from scipy.stats import spearmanr

In [2]:
#Import required sklearn functions
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import VarianceThreshold
from collections import defaultdict
from sklearn.preprocessing import normalize

In [3]:
#Import sklearn classifiers
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF, ConstantKernel
from sklearn.linear_model import LogisticRegression

In [4]:
#Import library to oversample 
from imblearn.over_sampling import RandomOverSampler

In [5]:
#Import RDKit and Mordred libraries
from rdkit import Chem
from rdkit.Chem import Draw
from mordred import Calculator, descriptors

In [6]:
#Allows figures to be visualized in jupyter notebook
%matplotlib inline

In [7]:
#Functions used in the study

#Remove those numbers from analysis data
def filter_rows_by_values1(df, col, values):
    return df[~df[col].isin(values)]

#Remove those numbers from analysis data
def filter_rows_by_values2(df, col, values):
    return df[df[col].isin(values)]

#Get Mordred calcs
def get_Mordred(data_input):
    # Assigns Reactants Mordred Info
    reactants = data_input['Substrate']
    
    reactants_mol_list = []
    for inChi_reactants in reactants:
      reactants_mol = Chem.MolFromInchi(inChi_reactants)
      reactants_mol_list.append(reactants_mol)

    # Puts reactants into Pandas Type
    reactant_data = []
    reactant_data = calc.pandas(reactants_mol_list)
       
    #Joins Mordred parameters with experimental, atomic charges, and JChem for Excel parameters
    add_reactants = pd.concat((data_input, reactant_data), axis=1)
    
    #Force any non-numeric entries as NaN and replace them with 0
    int_data = add_reactants.apply(pd.to_numeric, errors='coerce')
    
    output = int_data.fillna(0)#, inplace=True)

    return output

#Remove zero varience
def remove_zero_varience(values):
   sel = VarianceThreshold()
   _ = sel.fit(values)
   mask = sel.get_support()
   values = values.loc[:,mask] 
   return values

def remove_95correlated(correlated):
    #Remove any features that are greater than 95% correlated
    corr_matrix = correlated.corr()
    upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape),k=1).astype(np.bool))

    to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.95)]

    correlated = correlated.drop(to_drop, axis = 1)
    corr_matrix = correlated.corr()
    return correlated

def remove_nonimportant(X_values, y_values):
    # Specifys Random Forest and the Number of Trees, SelectFromModel will
    # select features which are most important
    feature_names = [f"feature {i}" for i in range(X_values.shape[1])]
    forest = RandomForestClassifier(random_state=42)
    forest.fit(X_values, y_values)

    start_time = time.time()
    importances = forest.feature_importances_
    std = np.std([tree.feature_importances_ for tree in forest.estimators_], axis=0)
    elapsed_time = time.time() - start_time

    threshold = np.sort(importances)[-100]
    
    sel = SelectFromModel(RandomForestClassifier(n_estimators = 800, max_depth=30),threshold=threshold)
    sel.fit(X_values, y_values)

    # Select the final features set 
    sel.get_support()
    selected_feat= X_values.columns[(sel.get_support())]

    # Prints the names of the final selected features
    print(selected_feat)
    X_values = X_values[selected_feat]
    
    return X_values

def dendrogram(X_values, y):
    corr = spearmanr(X_values).correlation
    # Ensure the correlation matrix is symmetric
    corr = (corr + corr.T) / 2
    np.fill_diagonal(corr, 1)
    distance_matrix = 1 - np.abs(corr)
    dist_linkage = hierarchy.ward(squareform(distance_matrix))
  
    trained_cluster_ids = hierarchy.fcluster(dist_linkage, y, criterion="distance")
    trained_cluster_id_to_feature_ids = defaultdict(list) 
    for idx, trained_cluster_id in enumerate(trained_cluster_ids):
        trained_cluster_id_to_feature_ids[trained_cluster_id].append(idx)
    
    trained_selected_features = [v[0] for v in trained_cluster_id_to_feature_ids.values()]
    final_selected_features = X_values.columns[trained_selected_features]
    X_train = X_values[final_selected_features]
    return X_train

def classificationMetrics(results, y_test, pred):
    acc = accuracy_score(y_test, pred)
    prec = precision_score(y_test, pred, average=None, zero_division=0)
    recall = recall_score(y_test, pred, average=None)
    F1 = f1_score(y_test, pred, average=None)
    comb = np.concatenate((x, y, acc, prec, recall, F1), axis=None)
    comb = [comb]
    results = results.append(pd.DataFrame(comb, columns=results.columns), ignore_index=True)
    return results


In [8]:
# Sets Pandas Display to Monitor Code
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 15)

In [9]:
# Create Mordred Calculator
calc = Calculator(descriptors, ignore_3D=True)

In [10]:
# Read Training/Test data input File
data = pd.read_csv('BorylationTrainingTest 1-10-25.csv')
data.head()

,Substrate,Product,Boronic Ester,Active Catalyst-Ligand,Catalyst,Ligand,Buried_Vol,...,Platt Index,Refractivity,Ring Atom Count,Rot Bond Count,Sigma Electronegativity,Wiener Index,Product_Ratio
0,"InChI=1S/C8H18/c1-3-5-7-8-6-4-2/h3-8H2,1-2H3",InChI=1S/C14H29BO2/c1-6-7-8-9-10-11-12-15-16-1...,2,6,1,1,43.1,...,12,38.6102,0,5,7.387931,84,1
1,"InChI=1S/C8H18/c1-3-5-7-8-6-4-2/h3-8H2,1-2H3",InChI=1S/C14H29BO2/c1-7-8-9-10-11-12(2)15-16-1...,2,6,1,1,43.1,...,12,38.6102,0,5,7.470532,84,0
2,"InChI=1S/C8H18/c1-3-5-7-8-6-4-2/h3-8H2,1-2H3",InChI=1S/C14H29BO2/c1-7-9-10-11-12(8-2)15-16-1...,2,6,1,1,43.1,...,12,38.6102,0,5,7.493894,84,0
3,"InChI=1S/C8H18/c1-3-5-7-8-6-4-2/h3-8H2,1-2H3",InChI=1S/C14H29BO2/c1-7-9-11-12(10-8-2)15-16-1...,2,6,1,1,43.1,...,12,38.6102,0,5,7.496149,84,0
4,"InChI=1S/C7H15N/c1-2-8-6-4-3-5-7-8/h2-7H2,1H3","InChI=1S/C13H26BNO2/c1-12(2)13(3,4)17-14(16-12...",2,7,3,1,41.0,...,18,36.8787,6,1,7.937482,64,1


In [11]:
#group the compounds by numbers
data['grouped'] = data.groupby('Substrate', sort=False).ngroup()
data[['Substrate','grouped']]

,Substrate,grouped
0,"InChI=1S/C8H18/c1-3-5-7-8-6-4-2/h3-8H2,1-2H3",0
1,"InChI=1S/C8H18/c1-3-5-7-8-6-4-2/h3-8H2,1-2H3",0
2,"InChI=1S/C8H18/c1-3-5-7-8-6-4-2/h3-8H2,1-2H3",0
3,"InChI=1S/C8H18/c1-3-5-7-8-6-4-2/h3-8H2,1-2H3",0
4,"InChI=1S/C7H15N/c1-2-8-6-4-3-5-7-8/h2-7H2,1H3",1
...,...,...
966,InChI=1S/C15H18/c1-10(2)13-7-5-11(3)14-8-6-12(...,188
967,InChI=1S/C15H18/c1-10(2)13-7-5-11(3)14-8-6-12(...,188
968,InChI=1S/C15H18/c1-10(2)13-7-5-11(3)14-8-6-12(...,188
969,InChI=1S/C15H18/c1-10(2)13-7-5-11(3)14-8-6-12(...,188


In [12]:
#Convert substrates to Mordred features
data = get_Mordred(data)

100%|██████████| 971/971 [01:27<00:00, 11.12it/s]


In [13]:
MLPResults_df = pd.DataFrame(columns = ['x', 'y', 'acc', 'precision 0',
                                   'precision 1','recall 0', 'recall 1', 'F1 0', 'F1 1'])

SVMResults_df = pd.DataFrame(columns =  ['x', 'y', 'acc', 'precision 0',
                                   'precision 1','recall 0', 'recall 1', 'F1 0', 'F1 1'])

GaussResults_df = pd.DataFrame(columns =  ['x', 'y', 'acc', 'precision 0',
                                   'precision 1','recall 0', 'recall 1', 'F1 0', 'F1 1'])

LRResults_df = pd.DataFrame(columns =  ['x', 'y', 'acc', 'precision 0',
                                   'precision 1','recall 0', 'recall 1', 'F1 0', 'F1 1'])

MLPmaxacc_comb = pd.DataFrame()
SVMmaxacc_comb = pd.DataFrame()
Gaussmaxacc_comb = pd.DataFrame()
LRmaxacc_comb = pd.DataFrame()

model_columns = pd.DataFrame()
for_range = range(1, 11)
for x in for_range:
    #Get numbers to represent compounds
    arr = np.arange(0, 188,  dtype=int)

    #Get 20% of numbers, without replacement
    set_numbers = np.random.choice(arr, int(len(arr)*0.20), replace=False ) 
    
    #Seperate training (80%) and test data (20%)
    training_data = filter_rows_by_values1(data, "grouped", set_numbers)
    training_data = training_data.drop('grouped', axis = 1)    
    test_data = filter_rows_by_values2(data, "grouped", set_numbers)
    test_data = test_data.drop('grouped', axis = 1)   

    # Seperate dataset as response variable (Product Ratio) and feature variables
    #Note: Product Ratio is described as "0" for non-borylating sites and "1" for borylating sites
    training_X = training_data.drop('Product_Ratio' , axis = 1)
    training_y = training_data['Product_Ratio']
    test_X = test_data.drop('Product_Ratio' , axis = 1)
    test_y = test_data['Product_Ratio']    
    
    training_X = pd.DataFrame(
    normalize(training_X),
    columns = training_X.columns)
    
    #Remove features that dont change
    training_X = remove_zero_varience(training_X)
    
    #Remove features that are more than 95% correlated
    training_X = remove_95correlated(training_X)
     
    #Remove features that are considered less important
    feature_names = [f"feature {i}" for i in range(training_X.shape[1])]
    forest = RandomForestClassifier(random_state=42)
    forest.fit(training_X, training_y)
    
    start_time = time.time()
    importances = forest.feature_importances_
    std = np.std([tree.feature_importances_ for tree in forest.estimators_], axis=0)
    elapsed_time = time.time() - start_time
    
    threshold = np.sort(importances)[-100] 
    sel = SelectFromModel(RandomForestClassifier(n_estimators = 800, max_depth=30),threshold=threshold)
    sel.fit(training_X, training_y)
     
    # Select the reduced features set 
    sel.get_support()
    selected_feat= training_X.columns[(sel.get_support())]
    
    reduced1_X = training_X[selected_feat]
    test_X = test_X[selected_feat]
    
    #Apply over-sampling to dataset
    ros = RandomOverSampler(random_state=10)
    X_resampled, y_resampled = ros.fit_resample(reduced1_X, training_y) 
    
    for y in [ 
              0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35,
              0.40, 0.40, 0.40, 0.40, 0.40, 0.40, 0.40, 0.40, 0.40, 0.40,
              0.45, 0.45, 0.45, 0.45, 0.45, 0.45, 0.45, 0.45, 0.45, 0.45,
              0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50,
             ]:
    
        #Make final training and test set and save them as df's  
        X_train = dendrogram(X_resampled, y)
        test_X = test_X[X_train.columns]
        training_columns_list = X_train.columns.tolist()
        training_columns_list = (x, y, training_columns_list)
        training_columns_list = pd.DataFrame(training_columns_list).T

        #MLP CLassifier
        MLP = MLPClassifier(alpha=1, max_iter=1000)
        MLP.fit(X_train, y_resampled)
        test_X_norm = pd.DataFrame(normalize(test_X), columns = test_X.columns)
        pred_MLP = MLP.predict(test_X_norm)
        MLPResults_df = classificationMetrics(MLPResults_df, test_y, pred_MLP)
        
        #Determine the mean accuracy of the different dendrogram settings
        acc_mean = MLPResults_df.groupby('y')['acc'].mean()
        acc_std = MLPResults_df.groupby('y')['acc'].std()
        precision_0_mean = MLPResults_df.groupby('y')['precision 0'].mean()
        precision_0_std = MLPResults_df.groupby('y')['precision 0'].std()        
        precision_1_mean = MLPResults_df.groupby('y')['precision 1'].mean()
        precision_1_std = MLPResults_df.groupby('y')['precision 1'].std()
        recall_0_mean = MLPResults_df.groupby('y')['recall 0'].mean()
        recall_0_std = MLPResults_df.groupby('y')['recall 0'].std()      
        recall_1_mean = MLPResults_df.groupby('y')['recall 1'].mean()
        recall_1_std = MLPResults_df.groupby('y')['recall 1'].std()
        F1_0_mean = MLPResults_df.groupby('y')['F1 0'].mean()
        F1_0_std = MLPResults_df.groupby('y')['F1 0'].std()
        F1_1_mean = MLPResults_df.groupby('y')['F1 1'].mean()
        F1_1_std = MLPResults_df.groupby('y')['F1 1'].std()
        
        MLPaverage_df = pd.concat([acc_mean , acc_std, 
                                   precision_0_mean, precision_0_std, 
                                   precision_1_mean, precision_1_std, 
                                   recall_0_mean, recall_0_std, 
                                   recall_1_mean, recall_1_std,
                                   F1_0_mean, F1_0_std,
                                   F1_1_mean, F1_1_std], axis=1)

        MLPaverage_df.columns = ['acc_mean' , 'acc_std', 'precision_0_mean', 'precision_0_std', 
                                 'precision_1_mean', 'precision_1_std', 'recall_0_mean', 'recall_0_std', 
                                 'recall_1_mean','recall_1_std', 'F1_0_mean', 'F1_0_std', 
                                 'F1_1_mean', 'F1_1_std']                                 
          
        
                
        MLPmaxacc = MLPaverage_df[MLPaverage_df.acc_mean == MLPaverage_df.acc_mean.max()]
        MLPmaxacc_copy  = MLPmaxacc.copy()
        MLPmaxacc_copy['x_col'] = x       
        
        #SVM CLassifier
        SVM = SVC(kernel="linear", C=1.0)
        SVM.fit(X_train, y_resampled)
        test_X_norm = pd.DataFrame(normalize(test_X), columns = test_X.columns)
        pred_SVM = SVM.predict(test_X_norm)
        SVMResults_df = classificationMetrics(SVMResults_df, test_y, pred_SVM)  
    
        #Determine the mean accuracy of the different dendrogram settings
        acc_mean = SVMResults_df.groupby('y')['acc'].mean()
        acc_std = SVMResults_df.groupby('y')['acc'].std()
        precision_0_mean = SVMResults_df.groupby('y')['precision 0'].mean()
        precision_0_std = SVMResults_df.groupby('y')['precision 0'].std()        
        precision_1_mean = SVMResults_df.groupby('y')['precision 1'].mean()
        precision_1_std = SVMResults_df.groupby('y')['precision 1'].std()
        recall_0_mean = SVMResults_df.groupby('y')['recall 0'].mean()
        recall_0_std = SVMResults_df.groupby('y')['recall 0'].std()      
        recall_1_mean = SVMResults_df.groupby('y')['recall 1'].mean()
        recall_1_std = SVMResults_df.groupby('y')['recall 1'].std()
        F1_0_mean = SVMResults_df.groupby('y')['F1 0'].mean()
        F1_0_std = SVMResults_df.groupby('y')['F1 0'].std()
        F1_1_mean = SVMResults_df.groupby('y')['F1 1'].mean()
        F1_1_std = SVMResults_df.groupby('y')['F1 1'].std()
        
        SVMaverage_df = pd.concat([acc_mean , acc_std, 
                                   precision_0_mean, precision_0_std, 
                                   precision_1_mean, precision_1_std, 
                                   recall_0_mean, recall_0_std, 
                                   recall_1_mean, recall_1_std,
                                   F1_0_mean, F1_0_std,
                                   F1_1_mean, F1_1_std], axis=1)

        SVMaverage_df.columns = ['acc_mean' , 'acc_std', 'precision_0_mean', 'precision_0_std', 
                                 'precision_1_mean', 'precision_1_std', 'recall_0_mean', 'recall_0_std', 
                                 'recall_1_mean','recall_1_std', 'F1_0_mean', 'F1_0_std', 
                                 'F1_1_mean', 'F1_1_std']                                 
                              
        SVMmaxacc = SVMaverage_df[SVMaverage_df.acc_mean == SVMaverage_df.acc_mean.max()]
        SVMmaxacc_copy  = SVMmaxacc.copy()
        SVMmaxacc_copy['x_col'] = x       
                
        #Gaussian Process CLassifier
        kernel =  ConstantKernel(constant_value=1,constant_value_bounds =(1e-5,1e6)) * RBF(length_scale=1e-1, length_scale_bounds=(1e-3, 1e3))
        Gauss = GaussianProcessClassifier(kernel = kernel)
        Gauss.fit(X_train, y_resampled)
        test_X_norm = pd.DataFrame(normalize(test_X), columns = test_X.columns)
        pred_Gauss = Gauss.predict(test_X_norm)
        GaussResults_df = classificationMetrics(GaussResults_df, test_y, pred_Gauss)
    
        #Determine the mean accuracy of the different dendrogram settings
        acc_mean = GaussResults_df.groupby('y')['acc'].mean()
        acc_std = GaussResults_df.groupby('y')['acc'].std()
        precision_0_mean = GaussResults_df.groupby('y')['precision 0'].mean()
        precision_0_std = GaussResults_df.groupby('y')['precision 0'].std()        
        precision_1_mean = GaussResults_df.groupby('y')['precision 1'].mean()
        precision_1_std = GaussResults_df.groupby('y')['precision 1'].std()
        recall_0_mean = GaussResults_df.groupby('y')['recall 0'].mean()
        recall_0_std = GaussResults_df.groupby('y')['recall 0'].std()      
        recall_1_mean = GaussResults_df.groupby('y')['recall 1'].mean()
        recall_1_std = GaussResults_df.groupby('y')['recall 1'].std()
        F1_0_mean = GaussResults_df.groupby('y')['F1 0'].mean()
        F1_0_std = GaussResults_df.groupby('y')['F1 0'].std()
        F1_1_mean = GaussResults_df.groupby('y')['F1 1'].mean()
        F1_1_std = GaussResults_df.groupby('y')['F1 1'].std()
        
        Gaussaverage_df = pd.concat([acc_mean , acc_std, 
                                   precision_0_mean, precision_0_std, 
                                   precision_1_mean, precision_1_std, 
                                   recall_0_mean, recall_0_std, 
                                   recall_1_mean, recall_1_std,
                                   F1_0_mean, F1_0_std,
                                   F1_1_mean, F1_1_std], axis=1)

        Gaussaverage_df.columns = ['acc_mean' , 'acc_std', 'precision_0_mean', 'precision_0_std', 
                                 'precision_1_mean', 'precision_1_std', 'recall_0_mean', 'recall_0_std', 
                                 'recall_1_mean','recall_1_std', 'F1_0_mean', 'F1_0_std', 
                                 'F1_1_mean', 'F1_1_std']                                 

        Gaussmaxacc = Gaussaverage_df[Gaussaverage_df.acc_mean == Gaussaverage_df.acc_mean.max()]
        Gaussmaxacc_copy  = Gaussmaxacc.copy()
        Gaussmaxacc_copy['x_col'] = x
        
        #Logistic CLassifier
        clf = LogisticRegression(solver = 'liblinear', C=0.5)
        clf.fit(X_train, y_resampled)
        test_X_norm = pd.DataFrame(normalize(test_X), columns = test_X.columns)
        pred_LR = clf.predict(test_X_norm)
        LRResults_df = classificationMetrics(LRResults_df, test_y, pred_LR)
        
        
        #Determine the mean accuracy of the different dendrogram settings
        acc_mean = LRResults_df.groupby('y')['acc'].mean()
        acc_std = LRResults_df.groupby('y')['acc'].std()
        precision_0_mean = LRResults_df.groupby('y')['precision 0'].mean()
        precision_0_std = LRResults_df.groupby('y')['precision 0'].std()        
        precision_1_mean = LRResults_df.groupby('y')['precision 1'].mean()
        precision_1_std = LRResults_df.groupby('y')['precision 1'].std()
        recall_0_mean = LRResults_df.groupby('y')['recall 0'].mean()
        recall_0_std = LRResults_df.groupby('y')['recall 0'].std()      
        recall_1_mean = LRResults_df.groupby('y')['recall 1'].mean()
        recall_1_std = LRResults_df.groupby('y')['recall 1'].std()
        F1_0_mean = LRResults_df.groupby('y')['F1 0'].mean()
        F1_0_std = LRResults_df.groupby('y')['F1 0'].std()
        F1_1_mean = LRResults_df.groupby('y')['F1 1'].mean()
        F1_1_std = LRResults_df.groupby('y')['F1 1'].std()
        
        LRaverage_df = pd.concat([acc_mean , acc_std, 
                                   precision_0_mean, precision_0_std, 
                                   precision_1_mean, precision_1_std, 
                                   recall_0_mean, recall_0_std, 
                                   recall_1_mean, recall_1_std,
                                   F1_0_mean, F1_0_std,
                                   F1_1_mean, F1_1_std], axis=1)

        LRaverage_df.columns = ['acc_mean' , 'acc_std', 'precision_0_mean', 'precision_0_std', 
                                 'precision_1_mean', 'precision_1_std', 'recall_0_mean', 'recall_0_std', 
                                 'recall_1_mean','recall_1_std', 'F1_0_mean', 'F1_0_std', 
                                 'F1_1_mean', 'F1_1_std']                                 

        LRmaxacc = LRaverage_df[LRaverage_df.acc_mean == LRaverage_df.acc_mean.max()]
        LRmaxacc_copy  = LRmaxacc.copy()
        LRmaxacc_copy['x_col'] = x                

    MLPmaxacc_comb = MLPmaxacc_comb.append(MLPmaxacc_copy)
    SVMmaxacc_comb = SVMmaxacc_comb.append(SVMmaxacc_copy)
    Gaussmaxacc_comb = Gaussmaxacc_comb.append(Gaussmaxacc_copy)
    LRmaxacc_comb = LRmaxacc_comb.append(LRmaxacc_copy)

#Write the results onto a CSV file, currently commented out 
MLPmaxacc_comb.to_csv("10Runs_ClassificationScreening_SVM_NN_GP_LR.csv",)
SVMmaxacc_comb.to_csv("10Runs_ClassificationScreening_SVM_NN_GP_LR.csv", mode="a")
Gaussmaxacc_comb.to_csv("10Runs_ClassificationScreening_SVM_NN_GP_LR.csv", mode="a")
LRmaxacc_comb.to_csv("10Runs_ClassificationScreening_SVM_NN_GP_LR.csv", mode="a")


In [14]:
MLPmaxacc_comb

,acc_mean,acc_std,precision_0_mean,precision_0_std,precision_1_mean,precision_1_std,recall_0_mean,recall_0_std,recall_1_mean,recall_1_std,F1_0_mean,F1_0_std,F1_1_mean,F1_1_std,x_col
y,,,,,,,,,,,,,,,
0.35,0.801961,0.019771,0.826860,0.005431,0.394638,0.082734,0.959281,0.039900,0.091892,0.071165,0.887695,0.014296,0.130283,0.078425,1
0.35,0.804122,0.013786,0.816571,0.011198,0.197319,0.210300,0.979641,0.034503,0.045946,0.067979,0.890224,0.010175,0.065141,0.085907,2
0.35,0.800380,0.012914,0.811914,0.011667,0.213854,0.209503,0.980491,0.030405,0.045946,0.062342,0.887883,0.009427,0.067994,0.082260,3
0.40,0.789322,0.062218,0.814629,0.014769,0.236667,0.210047,0.960492,0.108487,0.066216,0.138212,0.876980,0.059634,0.071736,0.089350,4
0.40,0.786892,0.056048,0.813431,0.013519,0.251660,0.195926,0.957373,0.097877,0.075135,0.126684,0.875824,0.053447,0.089176,0.091885,5
0.35,0.789357,0.045283,0.811749,0.010562,0.193926,0.191336,0.963388,0.080381,0.063964,0.105555,0.878876,0.038979,0.077831,0.091995,6
0.40,0.791306,0.048006,0.812105,0.011899,0.198378,0.198591,0.966414,0.084625,0.060232,0.112539,0.879784,0.045633,0.073257,0.089875,7
0.45,0.791003,0.070848,0.799916,0.090869,0.181242,0.190256,0.968253,0.113623,0.052365,0.120971,0.875795,0.099823,0.063294,0.081832,8
0.45,0.791512,0.066765,0.799435,0.085623,0.161104,0.188178,0.971780,0.107518,0.046547,0.115168,0.876946,0.094105,0.056261,0.079650,9


In [15]:
SVMmaxacc_comb

,acc_mean,acc_std,precision_0_mean,precision_0_std,precision_1_mean,precision_1_std,recall_0_mean,recall_0_std,recall_1_mean,recall_1_std,F1_0_mean,F1_0_std,F1_1_mean,F1_1_std,x_col
y,,,,,,,,,,,,,,,
0.40,0.818627,0.000000,0.821782,0.000000,0.500000,0.000000,0.994012,0.000000,0.027027,0.000000,0.899729,0.000000,0.051282,0.000000,1
0.45,0.818627,0.000000,0.821782,0.000000,0.500000,0.000000,0.994012,0.000000,0.027027,0.000000,0.899729,0.000000,0.051282,0.000000,1
0.50,0.818627,0.000000,0.821782,0.000000,0.500000,0.000000,0.994012,0.000000,0.027027,0.000000,0.899729,0.000000,0.051282,0.000000,1
0.40,0.812455,0.006333,0.814032,0.007951,0.250000,0.256495,0.997006,0.003072,0.013514,0.013865,0.896241,0.003578,0.025641,0.026307,2
0.45,0.812455,0.006333,0.814032,0.007951,0.250000,0.256495,0.997006,0.003072,0.013514,0.013865,0.896241,0.003578,0.025641,0.026307,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0.50,0.798650,0.016386,0.810965,0.005871,0.285307,0.174568,0.978392,0.026998,0.047297,0.048557,0.886659,0.011743,0.074883,0.064614,8
0.40,0.798309,0.015468,0.808888,0.007207,0.246199,0.182840,0.981505,0.026414,0.039039,0.048128,0.886674,0.011086,0.061537,0.064643,9
0.45,0.798309,0.015468,0.809256,0.007365,0.253606,0.187563,0.980793,0.026337,0.042042,0.048128,0.886603,0.011065,0.066563,0.065314,9


In [16]:
Gaussmaxacc_comb

,acc_mean,acc_std,precision_0_mean,precision_0_std,precision_1_mean,precision_1_std,recall_0_mean,recall_0_std,recall_1_mean,recall_1_std,F1_0_mean,F1_0_std,F1_1_mean,F1_1_std,x_col
y,,,,,,,,,,,,,,,
0.40,0.789216,0.000000,0.833333,0.000000,0.333333,0.000000,0.928144,0.000000,0.162162,0.000000,0.878187,0.000000,0.218182,0.000000,1
0.45,0.789216,0.000000,0.833333,0.000000,0.333333,0.000000,0.928144,0.000000,0.162162,0.000000,0.878187,0.000000,0.218182,0.000000,1
0.50,0.789216,0.000000,0.833333,0.000000,0.333333,0.000000,0.928144,0.000000,0.162162,0.000000,0.878187,0.000000,0.218182,0.000000,1
0.40,0.797749,0.008755,0.819808,0.013877,0.166667,0.170996,0.964072,0.036861,0.081081,0.083187,0.885470,0.007473,0.109091,0.111925,2
0.45,0.797749,0.008755,0.819808,0.013877,0.166667,0.170996,0.964072,0.036861,0.081081,0.083187,0.885470,0.007473,0.109091,0.111925,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0.35,0.678841,0.216878,0.682188,0.307770,0.231744,0.110620,0.771894,0.352240,0.274775,0.338745,0.723400,0.326992,0.187844,0.114165,6
0.35,0.609251,0.264002,0.584733,0.372566,0.226025,0.103259,0.661623,0.424389,0.378378,0.404294,0.620057,0.395519,0.206972,0.115636,7
0.35,0.558888,0.280810,0.594974,0.349255,0.221679,0.097194,0.582210,0.449458,0.449324,0.422425,0.548879,0.415387,0.220873,0.114232,8


In [17]:
LRmaxacc_comb

,acc_mean,acc_std,precision_0_mean,precision_0_std,precision_1_mean,precision_1_std,recall_0_mean,recall_0_std,recall_1_mean,recall_1_std,F1_0_mean,F1_0_std,F1_1_mean,F1_1_std,x_col
y,,,,,,,,,,,,,,,
0.40,0.818627,0.000000,0.821782,0.000000,0.500000,0.000000,0.994012,0.000000,0.027027,0.000000,0.899729,0.000000,0.051282,0.000000,1
0.45,0.818627,0.000000,0.821782,0.000000,0.500000,0.000000,0.994012,0.000000,0.027027,0.000000,0.899729,0.000000,0.051282,0.000000,1
0.50,0.818627,0.000000,0.821782,0.000000,0.500000,0.000000,0.994012,0.000000,0.027027,0.000000,0.899729,0.000000,0.051282,0.000000,1
0.40,0.812455,0.006333,0.814032,0.007951,0.250000,0.256495,0.997006,0.003072,0.013514,0.013865,0.896241,0.003578,0.025641,0.026307,2
0.45,0.812455,0.006333,0.814032,0.007951,0.250000,0.256495,0.997006,0.003072,0.013514,0.013865,0.896241,0.003578,0.025641,0.026307,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0.45,0.794131,0.015878,0.812624,0.005490,0.316259,0.136966,0.967936,0.026847,0.067568,0.049031,0.883318,0.011417,0.104568,0.065701,8
0.50,0.794131,0.015878,0.812624,0.005490,0.316259,0.136966,0.967936,0.026847,0.067568,0.049031,0.883318,0.011417,0.104568,0.065701,8
0.45,0.794292,0.014967,0.810730,0.007468,0.281119,0.163222,0.971499,0.027248,0.060060,0.050891,0.883633,0.010794,0.092949,0.070169,9
